<a href="https://colab.research.google.com/github/andreidm92/computer_vision_tasks/blob/main/practice/Tracking_objects_Calman_Sort_Lesson3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## 🧠 Что такое "трек" в трекинге?

**Трек (track)** — это объект, отслеживаемый во времени: его ID, координаты, скорость, статус (активен, потерян, удалён).

**Цикл жизни трека:**

- 🔵 Новый объект обнаружен → создаётся новый трек (инициализация)
- 🟢 Каждый следующий кадр: если детекция найдена рядом — трек обновляется
- 🟡 Если детекция пропала — трек переходит в статус "потерян"
- 🔴 Если объект не появляется N кадров подряд — трек удаляется

Состояние трека в DeepSORT:
- Confirmed — трек существует и стабилен
- Tentative — только что создан, ждет подтверждения
- Deleted — удален, больше не используется

Треки обеспечивают: стабильные ID, восстановление после occlusion, прогноз движения.



## 📐 Kalman фильтр: Основы

Kalman фильтр — это метод оценки состояния динамической системы с учетом шума. В CV он предсказывает положение объектов.

Состояние объекта:

$$
\mathbf{x} = \begin{bmatrix}
x \\
y \\
s \\
r \\
\dot{x} \\
\dot{y} \\
\dot{s}
\end{bmatrix}
$$

Процесс предсказания:

$$
\mathbf{x}_{t|t-1} = \mathbf{F} \mathbf{x}_{t-1|t-1}
$$

Обновление по измерению:

$$
\mathbf{x}_{t|t} = \mathbf{x}_{t|t-1} + \mathbf{K}(\mathbf{z}_t - \mathbf{H} \mathbf{x}_{t|t-1})
$$

Где:

- $\mathbf{F}$ — матрица перехода  
- $\mathbf{z}_t$ — измерение (например, bbox от YOLO)  
- $\mathbf{H}$ — матрица наблюдения  
- $\mathbf{K}$ — матрица Калмана (вес измерения vs предсказания)



## 🔍 Пошаговый пример работы Венгерского алгоритма

Пусть у нас есть матрица стоимости:

$$
C = \begin{bmatrix}
4 & 2 & 3 \\
1 & 0 & 2 \\
3 & 5 & 2
\end{bmatrix}
$$

### Шаг 1: Вычитаем минимум по строкам

- Строка 1: $[4,2,3] \rightarrow [2,0,1]$  
- Строка 2: $[1,0,2] \rightarrow [1,0,2]$  
- Строка 3: $[3,5,2] \rightarrow [1,3,0]$

Матрица после вычитания по строкам:

$$
\begin{bmatrix}
2 & 0 & 1 \\
1 & 0 & 2 \\
1 & 3 & 0
\end{bmatrix}
$$

### Шаг 2: Вычитаем минимум по столбцам

- Столбец 1: минимум = 1 → $[2,1,1] \rightarrow [1,0,0]$  
- Столбец 2: минимум = 0  
- Столбец 3: минимум = 0

После двух вычитаний:

$$
\begin{bmatrix}
1 & 0 & 1 \\
0 & 0 & 2 \\
0 & 3 & 0
\end{bmatrix}
$$

### Шаг 3: Покрываем нули минимальным числом линий

Покрываем:
- Строку 2 (нулей 2)
- Столбец 1 (нулей 2)
- Столбец 3 (1 ноль)

→ Используем $3$ линии, что соответствует размеру матрицы $3 \times 3$ → можно переходить к построению назначений.

**Результат назначения:**
- Трек 1 → Детекция 2  
- Трек 2 → Детекция 1  
- Трек 3 → Детекция 3  
Суммарная стоимость: **2 + 1 + 2 = 5**


# 🛠️ ЧАСТЬ 2: Практика — YOLOv8 + SORT в Google Colab

In [ ]:
# Установка библиотек
!pip install ultralytics filterpy opencv-python deep_sort_realtime --quiet

In [ ]:
# 📥 Скачивание SORT
!git clone https://github.com/abewley/sort.git
%cd sort


In [ ]:
# 1) Пути к файлам
input_path  = '/content/Vence.MP4'
output_path = '/content/output_sort.mp4'

# 2) Импорт и инициализация
import cv2
import torch
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
from IPython.display import HTML
from base64 import b64encode

In [ ]:
# 3) Загрузка модели YOLO и отправка на GPU
model = YOLO('yolov8n.pt').to('cuda')

# 4) Инициализация трекера DeepSORT с использованием GPU
tracker = DeepSort(max_age=30, embedder_gpu=True)

# 5) Захват видео и настройки
cap = cv2.VideoCapture(input_path)
fps = cap.get(cv2.CAP_PROP_FPS)
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (w, h))

# 6) Основной цикл обработки кадров
frame_id = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_id += 1
    if frame_id % 30 == 0:
        print(f'Processing frame {frame_id}')

    # 6.1) Детекция объектов с YOLOv8
    with torch.no_grad():
        results = model(frame)[0]

    # 6.2) Подготовка детекций для DeepSORT
    detections = []
    for *box, conf, cls in results.boxes.data.cpu().numpy():
        if conf < 0.5:
            continue
        x1, y1, x2, y2 = map(int, box)
        w_box, h_box = x2 - x1, y2 - y1
        detections.append(([x1, y1, w_box, h_box], float(conf), int(cls)))

    # 6.3) Обновление трекера
    tracks = tracker.update_tracks(detections, frame=frame)

    # 6.4) Отрисовка траекторий объектов
    for track in tracks:
        if not track.is_confirmed():
            continue
        l, t, r, b = map(int, track.to_ltrb())
        tid = track.track_id
        cv2.rectangle(frame, (l, t), (r, b), (0, 255, 0), 2)
        cv2.putText(frame, f'ID: {tid}', (l, t - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    # 6.5) Сохранение кадра
    out.write(frame)

# 7) Завершение работы
cap.release()
out.release()

# 8) Отображение видео внутри Colab
with open(output_path, 'rb') as f:
    mp4 = f.read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f"""<video width=640 controls>
           <source src="{data_url}" type="video/mp4">
         </video>""")


#🛠️ ЧАСТЬ 3: Фильтр Калмана


In [ ]:
from filterpy.kalman import KalmanFilter
import numpy as np
import matplotlib.pyplot as plt

# Инициализация Kalman фильтра
kf = KalmanFilter(dim_x=2, dim_z=1)
kf.x = np.array([0., 0.])
kf.F = np.array([[1,1],[0,1]])
kf.H = np.array([[1,0]])
kf.P *= 1000.
kf.R = 5
kf.Q = 0.1 * np.eye(2)

np.random.seed(0)
zs = [i + np.random.randn()*5 for i in range(50)]

predictions = []
for z in zs:
    kf.predict()
    kf.update(z)
    predictions.append(kf.x[0])

plt.figure(figsize=(10,4))
plt.plot(zs, label='Измерения')
plt.plot(predictions, label='Kalman предсказания')
plt.title("1D Kalman Tracking")
plt.xlabel("Кадры")
plt.ylabel("Позиция")
plt.grid()
plt.legend()
plt.show()
